# Teste Simples - Lead Qualification Agent

Notebook direto e enxuto para testar o agente de qualificação de leads via API.

In [29]:
import requests
import json
import uuid

# Configuração
BASE_URL = "http://localhost:8000"
APP_NAME = "lead_qualification_agent"

In [26]:
## Aqui você cria a session e mantém.

def qualify_lead(lead_data: str, user_id: str = "u_test") -> dict:
    """Qualifica um lead e retorna apenas o essencial."""
    session_id = f"s_{uuid.uuid4().hex[:8]}"
    
    # 1. Criar sessão
    requests.post(
        f"{BASE_URL}/apps/{APP_NAME}/users/{user_id}/sessions/{session_id}",
        json={"state": {}, "user_id": user_id, "session_id": session_id}
    )
    
    # 2. Executar agente
    requests.post(
        f"{BASE_URL}/run",
        json={
            "appName": APP_NAME,
            "userId": user_id,
            "sessionId": session_id,
            "newMessage": {"parts": [{"text": lead_data}], "role": "user"}
        }
    )
    
    # 3. Obter resultado
    response = requests.get(
        f"{BASE_URL}/apps/{APP_NAME}/users/{user_id}/sessions/{session_id}"
    )
    state = response.json().get("state", {})
    
    return {
        "validacao": state.get("validation_status", "N/A"),
        "score": state.get("lead_score", "N/A"),
        "recomendacao": state.get("action_recommendation", "N/A")
    }
    
## Aqui você cria a session e deleta ela depois -> Não quero gerenciar sessões manuais.

def qualify_lead_without_session(lead_data: str, user_id: str = "u_test") -> dict:
    """Qualifica um lead e retorna apenas o essencial."""
    session_id = f"s_{uuid.uuid4().hex[:8]}"
    
    # 1. Criar sessão
    requests.post(
        f"{BASE_URL}/apps/{APP_NAME}/users/{user_id}/sessions/{session_id}",
        json={"state": {}, "user_id": user_id, "session_id": session_id}
    )
    
    # 2. Executar agente
    requests.post(
        f"{BASE_URL}/run",
        json={
            "appName": APP_NAME,
            "userId": user_id,
            "sessionId": session_id,
            "newMessage": {"parts": [{"text": lead_data}], "role": "user"}
        }
    )
    
    # 3. Obter resultado
    response = requests.get(
        f"{BASE_URL}/apps/{APP_NAME}/users/{user_id}/sessions/{session_id}"
    )
    
    # 4. Deletar sessão
    requests.delete(
        f"{BASE_URL}/apps/{APP_NAME}/users/{user_id}/sessions/{session_id}"
    )
    
    state = response.json().get("state", {})
    
    return {
        "validacao": state.get("validation_status", "N/A"),
        "score": state.get("lead_score", "N/A"),
        "recomendacao": state.get("action_recommendation", "N/A")
    }

## Casos de Teste

In [28]:
# Teste 1: Lead de alta qualidade
lead1 = """Nome: Carlos Silva
Email: carlos.silva@techcorp.com.br
Cargo: Diretor de TI
Telefone: (11) 98765-4321
Empresa: TechCorp Ltda
Mensagem: Precisamos urgente de automação de vendas. Orçamento de R$ 50.000 aprovado. Implementação em 30 dias."""

# result = qualify_lead(lead1)
result = qualify_lead_without_session(lead1)
print("LEAD ALTA QUALIDADE")
print(f"Validação: {result['validacao']}")
print(f"Score: {result['score']}")
print(f"Ação: {result['recomendacao']}\n")

LEAD ALTA QUALIDADE
Validação: válido
Score: 10: Tomador de decisão com orçamento aprovado, necessidade urgente e cronograma definido.
Ação: Recomendação para a equipe de vendas:

Agendar imediatamente uma demonstração personalizada para Carlos Silva, Diretor de TI da TechCorp Ltda. Dada a urgência, orçamento aprovado e cronograma definido, preparar uma proposta detalhada alinhada às necessidades específicas da TechCorp e focada em um rápido retorno sobre o investimento. Priorizar este lead para fechamento imediato.



In [17]:
# Get in all sessions
def get_all_sessions(user_id: str = "u_test") -> list:
    """Obtém todas as sessões de um usuário."""
    response = requests.get(
        f"{BASE_URL}/apps/{APP_NAME}/users/{user_id}/sessions"
    )
    return response.json()

user_id = "u_test"
all_sessions = get_all_sessions(user_id=user_id)
print(f"Sessões ativas para o usuário {user_id}: {all_sessions}\n")

Sessões ativas para o usuário u_test: []



In [19]:
# Teste 2: Lead de baixa qualidade
lead2 = """Nome: João
Email: joao123@gmail.com
Mensagem: Quero saber mais sobre seus produtos."""

result = qualify_lead_without_session(lead2)
print("LEAD BAIXA QUALIDADE")
print(f"Validação: {result['validacao']}")
print(f"Score: {result['score']}")
print(f"Ação: {result['recomendacao']}\n")

LEAD BAIXA QUALIDADE
Validação: válido
Score: 2: Consulta geral sem detalhes de necessidade ou orçamento.
Ação: Recomendação: Para João, sugiro iniciar uma campanha de nutrição, focando em conteúdo educacional sobre nossos produtos e seus benefícios. O objetivo é fornecer mais informações e despertar o interesse em soluções específicas.



In [20]:
# Teste 3: Lead inválido (sem contato)
lead3 = """Empresa: Empresa ABC
Mensagem: Precisamos de CRM para 50 vendedores."""

result = qualify_lead_without_session(lead3)
print("LEAD INVÁLIDO")
print(f"Validação: {result['validacao']}")
print(f"Score: {result['score']}")
print(f"Ação: {result['recomendacao']}\n")

LEAD INVÁLIDO
Validação: válido
Score: 6: Necessidade expressa para um número definido de usuários.
Ação: Recomendação: Dada a necessidade expressa do lead e o número definido de usuários, sugiro uma chamada de descoberta para entender completamente suas necessidades e avaliar como nosso CRM pode atendê-las.



In [21]:
# Teste 4: Lead urgente
lead4 = """Nome: Dr. Roberto Mendes
Email: roberto.mendes@hospitalcentral.com.br
Telefone: (11) 3456-7890
Cargo: Diretor Administrativo
Empresa: Hospital Central
Mensagem: Sistema atual com falhas críticas. Orçamento de R$ 200.000 aprovado. Preciso implementar este trimestre."""

result = qualify_lead_without_session(lead4)
print("LEAD URGENTE")
print(f"Validação: {result['validacao']}")
print(f"Score: {result['score']}")
print(f"Ação: {result['recomendacao']}\n")

LEAD URGENTE
Validação: válido
Score: 9: Tomador de decisão com orçamento aprovado e necessidade urgente de implementação.
Ação: Recomendação para a equipe de vendas:

Considerando que o Dr. Roberto Mendes é o tomador de decisão, possui orçamento aprovado e tem uma necessidade urgente de implementação, sugiro agendar imediatamente uma demonstração do nosso produto. Prepare uma proposta detalhada que atenda às necessidades específicas do Hospital Central e avance para o fechamento da venda o mais rápido possível.



## Teste em Lote

In [22]:
from tqdm import tqdm

leads = {
    "Alta": lead1,
    "Baixa": lead2,
    "Inválido": lead3,
    "Urgente": lead4
}

print("RESUMO - QUALIFICAÇÃO EM LOTE\n")
print(f"{'Lead':<12} {'Validação':<20} {'Score':<40}")
print("-" * 72)

for nome, lead in tqdm(leads.items()):
    r = qualify_lead_without_session(lead)
    v = r['validacao'][:18] if len(r['validacao']) > 18 else r['validacao']
    s = r['score'][:38] if len(r['score']) > 38 else r['score']
    print(f"{nome:<12} {v:<20} {s:<40}")

RESUMO - QUALIFICAÇÃO EM LOTE

Lead         Validação            Score                                   
------------------------------------------------------------------------


 25%|██▌       | 1/4 [00:13<00:40, 13.61s/it]

Alta         válido               10: Tomador de decisão com orçamento a  


 50%|█████     | 2/4 [00:26<00:26, 13.40s/it]

Baixa        válido               2: Consulta genérica sem detalhes sobr  


 75%|███████▌  | 3/4 [00:39<00:12, 12.99s/it]

Inválido     válido               6: Necessidade declarada para um taman  


100%|██████████| 4/4 [00:52<00:00, 13.02s/it]

Urgente      válido               10: Tomador de decisão com orçamento a  


## API Direta (sem função helper)

In [23]:
# Exemplo de chamada direta
sid = f"s_{uuid.uuid4().hex[:8]}"
uid = "u_direct"

# 1. Criar sessão
requests.post(
    f"{BASE_URL}/apps/{APP_NAME}/users/{uid}/sessions/{sid}",
    json={"state": {}, "user_id": uid, "session_id": sid}
)

# 2. Executar
requests.post(
    f"{BASE_URL}/run",
    json={
        "appName": APP_NAME,
        "userId": uid,
        "sessionId": sid,
        "newMessage": {"parts": [{"text": lead1}], "role": "user"}
    }
)

# 3. Resultado
r = requests.get(f"{BASE_URL}/apps/{APP_NAME}/users/{uid}/sessions/{sid}")
state = r.json()["state"]

# 4. Deletar sessão
requests.delete(
    f"{BASE_URL}/apps/{APP_NAME}/users/{uid}/sessions/{sid}"
)

print("CHAMADA DIRETA À API")
print(f"Validação: {state.get('validation_status')}")
print(f"Score: {state.get('lead_score')}")

CHAMADA DIRETA À API
Validação: válido
Score: 10: Diretor de TI com orçamento aprovado e necessidade urgente de implementação.


In [24]:
all_sessions = get_all_sessions(user_id=uid)
print(f"Sessões ativas para o usuário {uid}: {all_sessions}\n")

Sessões ativas para o usuário u_direct: []



---

## Resumo

**Função principal:** `qualify_lead(lead_data, user_id)`

**Retorna:**
- `validacao` - Status de validação do lead
- `score` - Pontuação e justificativa
- `recomendacao` - Ação recomendada

**Fluxo:** Criar sessão → Executar agente → Obter state

**API:** `http://localhost:8000` (servidor ADK rodando)